<a href="https://colab.research.google.com/github/nekoniii3/Colabo_Samples/blob/main/022_Code_Interpreter/GPT_Code_Interpreter_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **準備：OpenAIパッケージのインストール**

In [ ]:
!pip install openai

# **①メッセージのみのやり取り（ファイル無し）**

### **1-1 OpenAI接続**

In [ ]:
from openai import OpenAI
import os

# APKIキーのセットとクライアント接続
os.environ["OPENAI_API_KEY"] = 'sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

client = OpenAI()

### **1-2 アシスタント作成 ※作成は1度で大丈夫です**

In [ ]:
assistant = client.beta.assistants.create(
  name="assistant01",
  instructions="あなたは優秀なアシスタントです。質問をされた場合は、質問に答えるコードを作成して実行します。",
  # model="gpt-4-1106-preview",
  model="gpt-3.5-turbo-1106",   # Code Interpreterはどのモデルでも大丈夫です
  tools=[{"type": "code_interpreter"}]
)

assistant_id = assistant.id

print(assistant_id)

### **※アシスタントが既にあればIDをセット**

In [ ]:
assistant_id = "asst_XXXXXXXXXXXXXXXXXXXXXXXXX"

### **1-3 スレッドの作成**

In [ ]:
# スレッドの作成
thread = client.beta.threads.create()

### **1-4 アシスタントへの問い合わせ**

In [ ]:
# 問い合わせ内容をセット
prompt = "こんにちは！"

# メッセージの作成
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
    # file_ids=[file_id]
)

In [ ]:
# RUNスタート（アシスタントへ問い合わせ）
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_id
  # instructions=""  # ここに新たにinstructionを入れることも可能
)

### **1-4 RUNの状態確認とメッセージの受け取り**

In [ ]:
# スレッドの実行状態を受け取る
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

# 状態を確認（completeで完了）
print(run.status)

In [ ]:
# メッセージを取得
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order = "asc",    # 昇順
  # after = "",   # これで前回以降のメッセージなど抽出が可能
)

In [ ]:
# メッセージの表示（※テキストのみ、画像がある場合エラーとなる恐れがあります）
for msg in messages:
    print(msg.role + "：" + msg.content[0].text.value)

# **②ファイルがある場合のやり取り**

### **※1-1～1-3は同じです<br>2-1 ファイルのアップ**

In [ ]:
# ファイルのアップ
file_response = client.files.create(
    purpose="assistants",
    file=open("ファイル名.csv","rb"),
)

# IDの取得
file_id = file_response.id

print(file_id)

### **2-2 ファイルをセットして、アシスタントへの問い合わせ**

In [ ]:
prompt = "これは○○のデータです。折れ線グラフでデータの可視化をお願いします。"

# メッセージの作成
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
    file_ids=[file_id]    # ※ファイルはLISTで複数も可能
)

In [ ]:
# スレッドの実行
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_id
  # instructions=""  # ここに新たにinstructionを入れることも可能
)

### **2-3 RUNの状態確認とメッセージの受け取り**

In [ ]:
# スレッドの実行状態を受け取る
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run.status)

In [ ]:
# メッセージを取得
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order = "asc"
)

In [ ]:
# メッセージの表示
for msg in messages:

    print(msg.role + "：")

    if msg.role == "assistant":

        for content in msg.content:

            image_fileid = ""
            assist_msg = ""

            cont_dict = content.model_dump()  # 辞書型に変換

            if cont_dict.get("image_file") is not None:

                image_fileid = cont_dict.get("image_file").get("file_id")

                print(f"{assist_msg}  ファイルID:{image_fileid}")

            if cont_dict.get("text") is not None:

                assist_msg = cont_dict.get("text").get("value")

                print(assist_msg)

    else:

        print(msg.content[0].text.value)




### **2-4 ファイルのダウンロード**

In [ ]:
# ファイルがある場合ダウンロード
image_fileid = 'file-Qi0N5wOJbz6S7oywOXS1LYHK'

retrieve_file = client.files.with_raw_response.retrieve_content(image_fileid)

content = retrieve_file.content

with open(file_id + ".png", 'wb') as f:
    f.write(content)

# **注釈ファイルの取得<br>**
## **※画像ファイル以外の取得**

### **※1-1～1-3は同じです<br>3-1 annotationsの取得**

In [ ]:
# メッセージの表示
for msg in messages:

    print(msg.role + "：")

    if msg.role == "assistant":

        for content in msg.content:

            image_fileid = ""
            assist_msg = ""

            cont_dict = content.model_dump()  # 辞書型に変換

            # 注釈（参照ファイル）ががある場合取得
            if len(cont_dict.get("text").get("annotations")) > 0:

                ct_ant = cont_dict.get("text").get("annotations")

                for ant in ct_ant:

                    if ant.get("file_path") is not None:

                        # 参照ファイルのID取得
                        ant_file_id = ct_ant[0].get("file_path").get("file_id")

                        if ant.get("text") is not None:

                            # ファイル形式（拡張子）取得
                            ext = "." + ct_ant[0].get("text")[ct_ant[0].get("text").rfind('.') + 1:]

                            print(f"file_id:{ant_file_id} ファイル形式:{ext}")

In [ ]:
ant_file_id = 'file-9reyyvBUPQiI8B7O2QPmZlGP'

ext = ".txt"

retrieve_file = client.files.with_raw_response.retrieve_content(ant_file_id)

content = retrieve_file.content

with open(file_id + ext, 'wb') as f:
    f.write(content)